In [9]:
import importlib
from CASutils import lensread_utils as lens
from CASutils import readdata_utils as read
from CASutils import calendar_utils as cal
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import sys

importlib.reload(lens)
importlib.reload(read)
importlib.reload(cal)

warnings.filterwarnings('ignore')

In [10]:
nmems=40
memstr = lens.lens1memnamegen(nmems)
preccdir="/project/mojave/cesm1/LENS/atm/month_1/PRECC/"
precldir="/project/mojave/cesm1/LENS/atm/month_1/PRECL/"

In [11]:
ybeg=1979 ; mbeg = 1 ; yend=2014 ; mend=12
nmonths=(yend-ybeg-1)*12 + (12-mbeg+1) + mend
datebeg = str(ybeg)+'-'+str(mbeg).zfill(2)
dateend = str(yend)+'-'+str(mend).zfill(2)

In [16]:
for imem in range(0,len(memstr),1):
    
    print(imem)
    
    # ----read in PRECC
    fhist=preccdir+"b.e11.B20TRC5CNBDRD.f09_g16."+memstr[imem]+".cam.h0.PRECC.*.nc"
    precchist = read.read_sfc_cesm(fhist,datebeg,"2005-12")
    
    frcp=preccdir+"b.e11.BRCP85C5CNBDRD.f09_g16."+memstr[imem]+".cam.h0.PRECC.*.nc"
    preccrcp = read.read_sfc_cesm(frcp,"2006-01",dateend)
    
    precc = xr.concat([precchist,preccrcp], dim="time", join="override")
    
    if (precc.time.size != nmonths):
        print("something's wrong.  nmonths="+str(nmonths)+" but got "+str(precc.time.size))
        sys.exit("Failed at PRECC for member "+memstr[imem])
    # ------------------
        
    # ----read in PRECL
    fhist=precldir+"b.e11.B20TRC5CNBDRD.f09_g16."+memstr[imem]+".cam.h0.PRECL.*.nc"
    preclhist = read.read_sfc_cesm(fhist,datebeg,"2005-12")
    
    frcp=precldir+"b.e11.BRCP85C5CNBDRD.f09_g16."+memstr[imem]+".cam.h0.PRECL.*.nc"
    preclrcp = read.read_sfc_cesm(frcp,"2006-01",dateend)
    
    precl = xr.concat([preclhist,preclrcp], dim="time", join="override")
    
    if (precl.time.size != nmonths):
        print("something's wrong.  nmonths="+str(nmonths)+" but got "+str(precc.time.size))
        sys.exit("Failed at PRECL for member "+memstr[imem])
    # ------------------
   
    precthist=precchist['PRECC'] + preclhist['PRECL']
    precthist = precthist.rename('PRECT')
    precthist = precthist.to_dataset()
    
    prectdjf = cal.season_ts(precthist,'PRECT','DJF')
    prectmam = cal.season_ts(precthist,'PRECT','MAM')
    prectjja = cal.season_ts(precthist,'PRECT','JJA')
    prectson = cal.season_ts(precthist,'PRECT','SON')
    
    if (imem == 0):
        djfgather = xr.DataArray(np.zeros([nmems, prectdjf.time.size, prectdjf.lat.size, prectdjf.lon.size]),
                                coords=[memstr, prectdjf['time'], prectdjf['lat'], prectdjf['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTDJF')
        
        mamgather = xr.DataArray(np.zeros([nmems, prectmam.time.size, prectmam.lat.size, prectmam.lon.size]),
                                coords=[memstr, prectmam['time'], prectmam['lat'], prectmam['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTMAM')
        
        jjagather = xr.DataArray(np.zeros([nmems, prectjja.time.size, prectjja.lat.size, prectjja.lon.size]),
                                coords=[memstr, prectjja['time'], prectjja['lat'], prectjja['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTJJA')
        
        songather = xr.DataArray(np.zeros([nmems, prectson.time.size, prectson.lat.size, prectson.lon.size]),
                                coords=[memstr, prectson['time'], prectson['lat'], prectson['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTSON')
        
    djfgather[imem,:,:,:] = np.array(prectdjf)*86400.*1000.
    mamgather[imem,:,:,:] = np.array(prectmam)*86400.*1000.
    jjagather[imem,:,:,:] = np.array(prectjja)*86400.*1000.
    songather[imem,:,:,:] = np.array(prectson)*86400.*1000.    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [17]:
outdir="/project/cas/islas/savs/drought/LENS2_vs_LENS1/outputdata/seasonalaverages/LENS1/"

djfgather.to_netcdf(path=outdir+"pr_djf.nc")
mamgather.to_netcdf(path=outdir+"pr_mam.nc")
jjagather.to_netcdf(path=outdir+"pr_jja.nc")
songather.to_netcdf(path=outdir+"pr_son.nc")